In [1]:
import torch

In [18]:
from utils import *
import pandas as pd
import numpy as np
import json

In [19]:
from datasets import load_dataset
import datasets
print(datasets.__version__)

2.16.1


In [20]:
data = load_dataset("sander-wood/irishman")

In [21]:
L=[] 
for i,x in enumerate(data['train']):
    L.append(x)
    if i>10: break
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(L, f)


In [7]:
data.save_to_disk("irishman")

Saving the dataset (0/1 shards):   0%|          | 0/214122 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2162 [00:00<?, ? examples/s]

In [8]:
sample = data['train'].to_pandas()

In [9]:
sample.shape

(214122, 2)

In [30]:
P = Patchilizer()

In [31]:
P.encode(sample['abc notation'])

AttributeError: 'Series' object has no attribute 'encode'

In [32]:
sample=list(sample['abc notation'].values)

In [34]:
encoded = np.array(P.encode(sample[0]))

In [35]:
encoded.shape

(22, 32)

In [37]:
len(P.split_bars(sample[0]))

18

In [41]:
for x in P.encode(sample[0]):
    print([chr(u) for u in x])

['\x01', 'X', ':', '1', '7', '6', '9', '9', '7', '\n', '\x02', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00']
['\x01', 'L', ':', '1', '/', '1', '6', '\n', '\x02', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00']
['\x01', 'M', ':', '4', '/', '4', '\n', '\x02', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00']
['\x01', 'K', ':', 'G', '\n', '\x02', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00', '\x00']
['\x01', ' ', 'D', '2', ' ', '|', ':', '\x02',

In [42]:
print(sample[0])

X:176997
L:1/16
M:4/4
K:G
 D2 |:"G" GFGA B2B2 |"G" GFGA B2B2 |"Am" dcBc A2A2 |"Am" dcBc"D7" A2A2 |"G" GFGA B2B2 | 
"G" GFGA B2B2 |"Am" dcBc"D7" AcBA |"G" G2B2 G4 :| Bc |:"G" d2>c2 B2d2 |"G" g2g2 d4 | 
"Am" dcBc A2A2 |"Am" dcBc"D7" A2A2 |"G" d2>c2 B2d2 |"G" g2g2 d4 |"Am" dcBc"D7" AcBA |"G" G2B2 G4 :|


In [43]:
P.decode(P.encode(sample[0]))

'X:176997\nL:1/16\nM:4/4\nK:G\n D2 |:"G" GFGA B2B2 |"G" GFGA B2B2 |"Am" dcBc A2A2 |"Am" dcBc"D7" A2A2 |"G" GFGA B2B2 | \n"G" GFGA B2B2 |"Am" dcBc"D7" AcBA |"G" G2B2 G4 :| Bc |:"G" d2>c2 B2d2 |"G" g2g2 d4 | \n"Am" dcBc A2A2 |"Am" dcBc"D7" A2A2 |"G" d2>c2 B2d2 |"G" g2g2 d4 |"Am" dcBc"D7" AcBA |"G" G2B2 G4 :|'

In [48]:
x=torch.nn.functional.one_hot(torch.tensor(P.encode(sample[0])),num_classes=128)

In [49]:
x.shape

torch.Size([22, 32, 128])

In [51]:
x.reshape(22,-1,32*128).shape

torch.Size([22, 1, 4096])

In [53]:
from transformers import GPT2Config

In [54]:

patch_config = GPT2Config(num_hidden_layers=PATCH_NUM_LAYERS, 
                    max_length=PATCH_LENGTH, 
                    max_position_embeddings=PATCH_LENGTH,
                    vocab_size=1)
char_config = GPT2Config(num_hidden_layers=CHAR_NUM_LAYERS, 
                    max_length=PATCH_SIZE, 
                    max_position_embeddings=PATCH_SIZE,
                    vocab_size=128)
model = TunesFormer(patch_config, char_config, share_weights=SHARE_WEIGHTS)

In [57]:
Y=model.patch_level_decoder(torch.tensor(P.encode(sample[0])))

In [64]:
torch.tensor(P.encode(sample[0])).shape

torch.Size([22, 32])

In [60]:
Y['last_hidden_state'].shape

torch.Size([22, 1, 768])

In [65]:
print(model.patch_level_decoder.base)

GPT2Model(
  (wte): Embedding(1, 768)
  (wpe): Embedding(128, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-8): 9 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)


In [66]:
print(model.char_level_decoder.base)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(128, 768)
    (wpe): Embedding(32, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-2): 3 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=128, bias=False)
)


In [67]:
pwd

'/home/jet08013/GitHub/tunesformer'